# NLP Classifer: Twitter Sentiment on Apple and Google Products (Phase 4 Project Submission)

* Student name: Konnor Clark
* Student pace: self paced
* Scheduled project review date/time: 05.22.2023
* Instructor name: Morgan Jones
* Blog post URL: https://datadruid.blogspot.com/

## Project Goal:



## Overview and Business Understanding/Questions



## Data Source and Exploration



In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('tweets.csv')
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,NaN,NaN,NaN
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,NaN,NaN,NaN
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,NaN,NaN,NaN
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,NaN,NaN,NaN
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,NaN,NaN,NaN


In [11]:
df = df.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis=1)

In [12]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [14]:
df.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

No emotion toward brand or product      5374
Positive emotion                        2976
Negative emotion                         570
I can't tell                             156
iPad                                       1
 Android                                   1
 Andro... {link} #sxsw #music #news"       1
 USA                                       1
 Texas                                     1
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [15]:
df.emotion_in_tweet_is_directed_at.value_counts()

iPad                                                      945
Apple                                                     661
iPad or iPhone App                                        469
Google                                                    430
iPhone                                                    297
Other Google product or service                           293
Android App                                                81
Android                                                    78
Other Apple product or service                             35
 iPad                                                       2
 please help."                                              2
 but it۪s coming this Frid... {link}"                       2
 March 14th {link}"                                         2
 TX                                                         1
 this is why it's good I'm not there #ooc"                  1
 singularity or #sxsw noobs meetup"                         1
 Possibl

#### Comments:
It seems that there are a few entries that have been mislabeled, perhaps shifted into an inappropriate column.  Since there are so few of these in comparison with the rest of the data, I am simply going to delete these mislabeled rows.

I also notice that the labels are subdivided within the larger categories of Apple and Google. There are also some repeat categories: "iPad," "iPad or iPhone App," and then another "iPad," perhaps with additional whitespace.  While these labels may help the model determine if a comment is negative or positive (perhaps a comment about the iPhone is more likely to be negative, for example), there is also ambiguity regarding the distinction in these categories.

For example, the distinction between 'iPad', 'iPad or iPhone App," and 'iPhone,' or between 'Google' and 'Other Google product or service.'  

I am going to decide to merge these into their larger categories to make things simpler so that every comment will be under the label Apple or Google.    

## Importing, Previewing, and Exploring Data


## Manipulating the Data

## Conclusion and Recommendation